# Cross-Validation (CV) and Bootstrap Examples in R

We illustarte solution to two problems from the following book:
    
James, G., Witten, D., Hastie, T., and Tibshirani, R. (2013). An Introduction to Statistical Learning with Applications in R, New York, NY:Springer-Verlag. ISBN: 978-1461471370. Chapter 5 <a href=\"http://www.StatLearning.com\" target=\"_blank\">Visit the Book Website</a>.

## Exercise 5, pages 198 - 199

Logistic Regression was used in Chapter 4 (Classification) to predict the probability of default using income and balance on the **Default** data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis for reproducibility of results.

This problem has multiple parts.

### (a) 

Fit a logistic regression model that uses **income** and **balance** to predict **default**.

In [ ]:
# the Default dataset is in ISLR library
library(ISLR)

# simplifies syntatctic notation for referencing variables in the dataset
attach(Default)

# to ensure reproducibility, set seed for the random number generator
set.seed(1)

In [ ]:
# how many observations and how many variables?
dim(Default)

In [ ]:
# show structure of the dataset -- variable names, their data types and sample values
str(Default)

In [ ]:
# show first few rows of the dataset
head(Default)

In [ ]:
# build a logistic regreesion model using the full dataset
glm.model <- glm(default ~ income + balance, family = binomial, data = Default)

In [ ]:
# explore the model
summary(glm.model)

### (b)

Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

- Split the sample set into a training set and a validation set.

- Fit a multiple logistic regression model using only the training observations

- Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

In [ ]:
# illustration of syntax
dim(Default)[1]

In [ ]:
# illustration of syntax
dim(Default)[2]

In [ ]:
# sample training data from the full dataset 
train.indices = sample(dim(Default)[1], dim(Default)[1]/2)

In [ ]:
# look at a few train indices
head(train.indices)

In [ ]:
# extract training data
training.data <- Default[train.indices, ]

In [ ]:
# list a few rows of training data
head(training.data)

dim(training.data)

In [ ]:
# test dataset = whole dataset - training data
test.data <- Default[-train.indices, ]

In [ ]:
dim(test.data)

In [ ]:
# fit a (multivariate) logistic regression model using only the training dataset

model.1 <- glm(default ~ income + balance, training.data, family = "binomial")

# another way to specify data for the model
# model.1 <- glm(default ~ income + balance, data = Default, subset = train.indices, family = "binomial")

summary(model.1)

Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

In [ ]:
# before prediction, set all credit card defaults to "No". 
# Note: we will be predicting on half of the observations in the Default dataset
glm.pred = rep("No", dim(Default)[1]/2)

In [ ]:
head(glm.pred)

In [ ]:
# make predictions using the test dataset
glm.prob = predict(model.1, test.data, type = "response")

In [ ]:
head(glm.prob)

In [ ]:
# set credit card customer default to "Yes" if the posterior probability > 0.5
glm.pred[glm.prob > 0.5] = "Yes"

Compute the validation set error, which is the fraction of the observation in the validation set that are misclassified.

In [ ]:
# compute mean classification error
mean(glm.pred != test.data$default)

In [ ]:
# extract default column values from the test data 
expected.default = test.data$default

In [ ]:
# compute confusion matrix -- another way to assess classification accuracy
confusion.matrix = table(glm.pred, expected.default)

In [ ]:
# print confusion.matrix
confusion.matrix

### (c)

Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

It is convenient to extract the steps into a function and call the function three times.

In [ ]:
model.assessment = function() {

	# sample observations for training
	train.indices = sample(dim(Default)[1], dim(Default)[1]/2)

	# build the model using training data
	model.1 <- glm(default ~ income + balance, data = Default, subset = train.indices, family = "binomial")

	# before prediction, set all credit card defaults to "No". Note: we will be predicting on half of the observations in the Default dataset

	glm.pred = rep("No", dim(Default)[1]/2)

	# make predictions
	glm.prob = predict(model.1, Default[-train.indices,], type = "response")

	# set credit card customer default to "Yes", if the posterior probability > 0.5

	glm.pred[glm.prob > 0.5] = "Yes"

	# compute mean of classification accuracy
	return (mean(glm.pred != Default[-train.indices, ]$default))

}

In [ ]:
# call model.assessment() first time
model.assessment()

In [ ]:
# call model.assessment() second time
model.assessment()

In [ ]:
# call model.assessment() third time
model.assessment()

### (d)

Now consider a logistic regression model that predicts the probability of **default** using **income**, **balance**, and a dummy variable for **student**. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for **student** leads to a reduction in the test error rate.

In [ ]:
# randomly select half of observations in Default for training
train.indices = sample( dim(Default)[1], dim(Default)[1]/2 )

In [ ]:
# build a logistic regression model using the training data
glm.fit = glm(default ~ income + balance + student, data = Default, subset = train.indices, family = binomial)

In [ ]:
# before prediction, set all credit card defaults to "No"
glm.pred = rep("No", dim(Default)[1]/2)

In [ ]:
head(glm.pred)

In [ ]:
# assess model performance by predicting on the test dataset
glm.probs = predict(glm.fit, Default[-train.indices, ], type = "response")

In [ ]:
# set credit card customer default to "Yes", if the posterior probability > 0.5
glm.pred[glm.probs>.5] = "Yes"

In [ ]:
# compute mean classification accuracy
mean(glm.pred != Default[-train.indices, ]$default)

In [ ]:
# extract default column values from the test dataset
expected.default = Default[-train.indices, ]$default

In [ ]:
# compute confusion matrix -- another way to assess classification accuracy
confusion.matrix = table(glm.pred, expected.default)

In [ ]:
# print confusion.matrix
confusion.matrix

## Exercise 6, page 199

We will further explore the use of a logistic regression model to predict the probability of **default** using **income** and **balance** on the **Default** dataset. In particular, we will now compute estimates for the standard errors of the **income** and **balance** logistic regression coefficients in two different ways: 

1. using the bootstrap, and 

2. using the standard formula for computing the standard errors in the glm() function. 

Do not forget to set a random seed before beginning of the analysis.

This problem has multiple parts.

### (a)

Using the **summary()** and **glm()** functions, determine the estimated standard errors for the coefficients associated with **income** and **balance** in a multiple logistic regression model that uses both predictors.

In [ ]:
# the Default dataset is in ISLR library
library(ISLR)

In [ ]:
# for syntactic simplicity
attach(Default)

In [ ]:
# to ensure reproducibility, set seed
set.seed(1)

In [ ]:
# build the model on the full dataset
glm.model = glm(default ~ income + balance, data = Default, family = binomial)

In [ ]:
# examine the model
summary(glm.model)

In [ ]:
# look at the model coefficients
summary(glm.model)$coefficient[ , 1]

### (b)

Write a function, boot.fn(), that takes as input the **Default** dataset as well as an index of the observations, and that outputs the coefficient estimates for **income** and **balance** in the multiple logistic regression model.

In [ ]:
boot.fn = function(data, index)
  return(coef(glm(default ~ income + balance, data = Default, subset = index, family = binomial)))

### (c)

Use the boot() function together with the boot.fn() function to estimate the standard errors of the logistic regression coefficients for **income** and **balance**.

In [ ]:
# load the library
library(boot)

In [ ]:
boot(Default, boot.fn, 100)

### (d)

Comment on the estimated standard errors obtained using the glm() function and the bootstrap function.

Estimated standard errors are similar.

## Exercise 8, page 200

This problem involves performing cross-validation on a simulated data set, and has multiple parts.

### (a)

Generate a simulated data set as follows:

In [ ]:
set.seed(1)

In [ ]:
Y = rnorm(100)

In [ ]:
Y

In [ ]:
X = rnorm(100)

In [ ]:
X

In [ ]:
Y = X - 2*X^2 + rnorm(100)

In this data set, what is $n$ and what is $p$? Write out the model
used to generate the data in equation form.

$n = 100, \,\, p = 2$

$Y = X - 2 X^2 + \epsilon$

### (b)

Create a scatterplot of $X$ against $Y$. Comment on what you find.

In [ ]:
plot(X, Y)

In this quadratic plot, $X$ varies from -2 to 2, and $Y$ from -8 to 2.

Load boot library, create a dataframe consisting of $X$ and $Y$, and set a random seed. 

In [ ]:
library(boot)

Data = data.frame(X,Y)

set.seed(1)

Compute the **LOOCV** errors that result from fitting the following four models using least squares: 

(i) $Y = \beta_0 + \beta_1 X + \epsilon$

In [ ]:
glmModel = glm(Y~X)

In [ ]:
cv.glm(Data, glmModel)$delta

(ii) $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \epsilon$

In [ ]:
glmModel = glm(Y ~ poly(X, 2))

In [ ]:
cv.glm(Data, glmModel)$delta

(iii) $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \epsilon$

In [ ]:
glmModel = glm(Y ~ poly(X, 3))

In [ ]:
cv.glm(Data, glmModel)$delta

(iv) $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \beta_4 X^4 + \epsilon$

In [ ]:
glmModel = glm(Y ~ poly(X, 4))

In [ ]:
cv.glm(Data, glmModel)$delta

### (d)

Repeat (c) using another random seed, and report your results. Are your results the same as what you got in (c)? Why?

In [ ]:
set.seed(100)

(i) $Y = \beta_0 + \beta_1 X + \epsilon$

In [ ]:
glmModel = glm(Y~X)

In [ ]:
cv.glm(Data, glmModel)$delta

In [ ]:
summary(glmModel)

(ii) $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \epsilon$

In [ ]:
glmModel = glm(Y ~ poly(X, 2))

In [ ]:
cv.glm(Data, glmModel)$delta

In [ ]:
summary(glmModel)

(iii) $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \epsilon$

In [ ]:
glmModel = glm(Y ~ poly(X, 3))

In [ ]:
cv.glm(Data, glmModel)$delta

In [ ]:
summary(glmModel)

(iv) $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \beta_4 X^4 + \epsilon$

In [ ]:
glmModel = glm(Y ~ poly(X, 4))

In [ ]:
cv.glm(Data, glmModel)$delta

In [ ]:
summary(glmModel)

Results are exactly the same. LOOCV is same since it evaluates $n$ folds of a single observation.

### (e)

Which of the models in (c) had the smallest LOOCV error? Is this what you expected? Explain your answer.

The data fits a quadratic model. It is not surprising that the quadratic polynomial had the lowest LOOCV test error rate. This was expected because it matches the true form of $Y$.

### (f)

Comment on the statistical significance of the coefficient estimates that results from fitting each of the models in (c) using least squares. Do these results agree with the conclusions drawn based on the cross-validation results?

As seen from the results of **summary(glmModel)** commands above, p-values show statistical significance of linear and quadratic terms. This agrees with the CV results.

## Exercise 9, pages 201

This problem involves estimating standard error using the bootstrap, and has multiple parts.

We will use **Boston housing dataset**, which is packaged with **MASS** librray.

In [ ]:
install.packages("MASS")

In [ ]:
library(MASS)

In [ ]:
str(Boston)

In [ ]:
summary(Boston)

In [ ]:
attach(Boston)

### (a)

Based on this dataset, provide an estimate for the population mean of **medv**. Call this estimate $\hat{\mu}$.

In [ ]:
medvMean = mean(medv)

In [ ]:
medvMean

### (b)

Provide an estimate of the standard error of $\hat{\mu}$. Interpret this result.

Hint: Compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.

In [ ]:
medvErr = sd(medv) / sqrt(length(medv))

In [ ]:
medvErr

### (c)

Now estimate the standard error of $\hat{\mu}$ using the bootstrap. How does this compare to your answer from (b)?

In [ ]:
boot.fn = function(data, index) return(mean(data[index]))

In [ ]:
library(boot)

In [ ]:
bStrap = boot(medv, boot.fn, 1000)

In [ ]:
bStrap

This answer is similar to answer from (b) up to two significant digits.

### (d)

Based on your bootstrap estimate from (c), provide a 95% confidence interval for the mean of medv. Compare it to the results obtained using t.test(Boston$medv).

**Hint**: You can approximate a 95% confidence interval using the formula [$\hat{\mu} − 2 \cdot SE(\hat{\mu}$), $\hat{\mu} + 2 \cdot SE(\hat{\mu}$)]

In [ ]:
t.test(medv)

In [ ]:
temp <- c(bStrap$t0 - 2*0.4119, bStrap$t0 + 2*0.4119)
temp

In [ ]:
Bootstrap estimate only 0.02 away for the t.test estimate.

### (e)

Based on this data set, provide an estimate, $\hat{\mu}_{med}$, for the median value of *medv* in the population.

In [ ]:
medvMed = median(medv)

In [ ]:
medvMed

### (f)

We now would like to estimate the standard error of $\hat{\mu}_{med}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

In [ ]:
boot.fn = function(data, index) return(median(data[index]))

In [ ]:
boot(medv, boot.fn, 1000)

Standard error is small relative to the median value.

### (g)

Based on this dataset, provide an estimate for the tenth percentile of *medv* in Boston suburbs. Call this quantity $\hat{\mu}_{0.1}$. (You may use the **quantile()** function.)

medvTenth = quantile(medv, c(0.1))

medvTenth

### (h)

Use the bootstrap to estimate the standard error of $\hat{\mu}_{0.1}$. Comment on your findings.

In [ ]:
boot.fn = function(data, index) return(quantile(data[index], c(0.1)))

In [ ]:
boot(medv, boot.fn, 1000)

The standard error is small relative to tenth-percentile value.